Partición de los datos
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/predictive-analytics/blob/master/09-R-data-partition.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/predictive-analytics/blob/master/09-R-data-partition.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/predictive-analytics/blob/master/LICENSE)  
[Readme](https://github.com/jdvelasq/predictive-analytics/blob/master/readme.md)

# Definición del problema real

En el tutorial anterior se construyó un clasificador que tiene como objetivo reprobar o aprobar solicitudes de crédito. El gerente de proyecto ha solitado que se haga un análisis del desempeño del sistema cuando se encuentre en productivo.

# Definición del problema en términos de los datos

El problema radica en que los datos para entrenamiento y prueba fueron tomados de forma determinística, pero los resultados del desempeño del clasificador podrían variar si la partición de los datos se hace diferente. Más aún, en vez de realizar estimaciones puntuales de la métrica de error seleccionada, sería mucho más conveniente estimar su distribución de probabilidades. 

In [3]:
## Se carga la librería
library(C50)

## Lectura de los datos
credit <- read.csv("data/credit.csv")
credit$default <- factor(credit$default, labels=c("No", "Yes"))

# Solución

A continuación se presentan los casos de análisis cuya complejidad aumenta de forma progresiva.

### Caso 1 - Selección determinística.

In [4]:
## Se toman los primero 900 datos para entrenamiento
## y los 100 últimos para prueba
train_sample <- 1:900
str(train_sample)

 int [1:900] 1 2 3 4 5 6 7 8 9 10 ...


In [5]:
## Se entrena el modelo y se prueba
credit_train <- credit[train_sample, ]
credit_test  <- credit[-train_sample, ]
credit_model <- C5.0(credit_train[-17], credit_train$default)
credit_pred <- predict(credit_model, credit_test)
length(credit_test$default[credit_pred != credit_test$default])

[1] 34

In [6]:
## métricas de error
table(credit_test$default, credit_pred)

     credit_pred
      No Yes
  No  54  14
  Yes 20  12

### Caso 2 - Selección aleatoria.

In [8]:
## cuál es el resultado si las muestras de entrenamiento
## y prueba son seleccionadas aleatoriamente?
set.seed(12345)   # se reinicia la semilla del generador aleatorio
train_sample <- sample(1000, 900)
str(train_sample)

 int [1:900] 721 875 760 884 455 166 324 506 722 981 ...


In [9]:
## entrenamiento y pronóstico
credit_train <- credit[train_sample, ]
credit_test  <- credit[-train_sample, ]
credit_model <- C5.0(credit_train[-17], credit_train$default)
credit_pred <- predict(credit_model, credit_test)
length(credit_test$default[credit_pred != credit_test$default])

[1] 26

In [10]:
## Métricas de error.
## Note que el error calculado cambio.
## Es decir, las métricas son una cantidad aleatoria
## y varia de acuerdo a como se haga la partición
table(credit_test$default, credit_pred)

     credit_pred
      No Yes
  No  59  14
  Yes 12  15

### Caso 3 - Validación cruzada.

Como se observó en las dos soluciones anteriores, los resultados varían de acuerdo a como se partan los datos; más aún, no hay información sobre el desempeño real de un modelo en producción. En la práctica, los datos se suelen partir en tres conjuntos, tal como muestra la gráfica de abajo:


* Conjunto de calibración de parámetros del modelo.


* Conjunto de prueba, usado comunmente para determinar la complejidad del modelo o el valor óptimo de alguno de sus parámetros.


* Conjunto de pronóstico, en el que se intenta reproducir el comportamiento del modelo en productivo.

![alt text](images/data-partition.jpg)

En los casos 1 y 2 se dividio el conjunto de datos únicamente en los conjuntos de calibración y prueba. En el Caso 1 se realizó la partición de forma secuencial, tal como se presenta en la figura anterior, mientras que en el Caso 2 se realizó la partición de forma aleatoria.

Si se tiene en cuenta que hay muchas particiones aleatorias posibles, una mejor estimación de la matriz de confusión (o cualquier otro estadístico que se calcule para un conjunto de datos) podría ser tomando los valores esperados de cada métrica. Es decir, si se repite el experimento $N$ veces, se tendrían $N$ valores posibles para cada uno de los elementos de la matriz de confusión y por lo tanto se podría tener su valor medio. Esta sería una métrica mucho más apropiada.

Sin embargo, este método no funciona muy bien porque el mismo conjunto de patrones podría aparecer repetido muchas veces, llevando a estimaciones erróneas.

**Ejercicio.--** Explique la afirmación anterior.

### Caso 4 - Bootstrap.

El bootstrap se basa en el remuestreo de los datos para poder obtener una muestra del estadístico que se está calculado (por ejemplo, la cantidad de falsos positivos FP). 

Supoga que tiene una muestra de ocho ejemplos:


$$\{x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8\}$$

Una muestra bootstap se obtiene de la muestra original, seleccionando ocho elementos de forma aleatoria. Por ejemplo, una muestra bootstrap podría ser:

$$\{x_1, x_2, x_2, x_2, x_4, x_1, x_7, x_7\}$$

Nóte que los elementos pueden repetirse. Sobre cada muestra bootstrap obtenida, se realiza el proceso de cómputo y se obtiene el estadístico de interés. Si este procedimiento se repite 500 veces para calcular la cantidad de FP, se tenddrían 500 valores posibles de FP. Una estimación mucho mejor del valor de FP, sería calcular su valor promedio a partir de la muestra de 500 valores. Más aún, podría calcularse el histograma o la distribución de probabilidades de FP, lo cual es mucho más informativo sobre el desempeño del modelo.

**Ejercicio.--** Suponga que $N$ = 100 (cantidad de muestras bootstrap), y calcule los valores promedios para los cuatro elementos de la matriz de confusión (problema de los créditos riesgosos).

**Ejercicio.--** Grafique el histograma de frecuencias para cada uno de los elementos de la matriz de transición del ejercicio anterior.

**Ejercicio.--** Cuál es la función (o funciones) del lenguaje R para realizar bootstraping?

### Caso 5. K-fold cross validation

En este método, el conjunto de datos para entrenamiento (ajuste + prueba) es dividido en $K$ grupos. Este es un proceso iterativo que opera de la siguiente forma (véase la figura de abajo). 


* Se toma el grupo 1 como conjunto de datos de prueba (grupo rojo) y se entrena el modelo con los grupos restantes {2, ..., K} (grupo negro).


* Se toma el grupo 2 como conjunto de datos de prueba (grupo rojo) y se entrena el modelo con los grupos restantes {1, 3, ..., K} (grupo negro).


* Se continua de esta forma hasta que se usa el grupo K para prueba, mientras que se usan los grupos 1 hasta K-1 para entrenamiento.



![alt text](images/k-fold-crossval.jpg)

De esta forma, se tienen K valores posibles para el estadístico de interés. Usualmente se reporta su valor promedio.

Note que una mejor opción sería distribuir los datos en cada grupo de forma aleatoria.

In [ ]:
## Las librerías irr y caret tienen funciones especializadas
## para aplicar esta metodología
## install.packages('irr')
library(irr)
library(caret)

In [25]:
## se crean 10 grupos
folds <- createFolds(credit$default, k = 10)
str(folds)

also installing the dependency ‘lpSolve’




The downloaded binary packages are in
	/var/folders/yq/svn60mh123z6dzr3d4rjk_740000gn/T//Rtmpykflrs/downloaded_packages


Loading required package: lpSolve


List of 10
 $ Fold01: int [1:100] 23 51 85 91 92 97 101 106 125 136 ...
 $ Fold02: int [1:100] 9 20 26 27 37 41 58 68 76 82 ...
 $ Fold03: int [1:100] 2 5 6 8 11 13 19 32 34 40 ...
 $ Fold04: int [1:100] 4 10 28 48 78 83 120 132 134 142 ...
 $ Fold05: int [1:100] 21 63 74 87 103 107 108 112 119 121 ...
 $ Fold06: int [1:100] 22 29 31 36 39 42 45 57 59 64 ...
 $ Fold07: int [1:100] 12 14 17 33 35 43 44 52 62 65 ...
 $ Fold08: int [1:100] 7 15 16 18 25 46 67 72 84 86 ...
 $ Fold09: int [1:100] 3 49 50 53 56 60 70 71 98 102 ...
 $ Fold10: int [1:100] 1 24 30 38 47 61 81 105 109 116 ...


In [26]:
## Esta variable guarda los resultados para cada uno 
## de los 10 grupos.
## lapply aplica la función especificada a cada uno 
## de los grupos
cv_results <- lapply(folds, function(x) {
    
    # elimina el grupo del patron de entrenamiento
    credit_train <- credit[-x, ]
    
    # usa el grupo como conjunto de prueba
    credit_test <- credit[x, ]
    
    # entrena el modelo y pronostica
    credit_model <- C5.0(default ~ ., data = credit_train)
    credit_pred <- predict(credit_model, credit_test)
    credit_actual <- credit_test$default
    
    # calcula una métrica de error
    kappa <- kappa2(data.frame(credit_actual, credit_pred))$value
    
    # retorna el valor calculado
    return(kappa)
  })

# Se calcula kappa para cada uno de los grupos
str(cv_results)

List of 10
 $ Fold01: num 0.266
 $ Fold02: num 0.475
 $ Fold03: num 0.255
 $ Fold04: num 0.175
 $ Fold05: num 0.255
 $ Fold06: num 0.193
 $ Fold07: num 0.417
 $ Fold08: num 0.381
 $ Fold09: num 0.303
 $ Fold10: num 0.475


In [28]:
## Se reporta el promedio de kappa 
mean(unlist(cv_results))

[1] 0.3195432

**Ejercicio.--** Use este método con K=15 para estimar la distribución de los valores de la matriz de confusión y compare contra los otros métodos presentados.  

### Caso 6. Método leave-one-out

Este es el mismo método anterior con K=1.

![alt text](images/leave-one-out.jpg)

---

Partición de los datos
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/predictive-analytics/blob/master/09-R-data-partition.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/predictive-analytics/blob/master/09-R-data-partition.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/predictive-analytics/blob/master/LICENSE)  
[Readme](https://github.com/jdvelasq/predictive-analytics/blob/master/readme.md)